In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import random
from scipy import stats

In [5]:
from scipy.stats import hypergeom


# Task 2


Условие: Было проведено секвенирование 15 образцов крови для детектирования
наследственных генетических вариантов в генах BRCA1, BRCA2, ATM, CHEK2. Суммарно для каждого 
образца покрыто 40000 точек генома. Покрытие каждой точки для каждого образца составило не менее 
100х (то есть, каждая геномная координата из 40000 покрыта не менее одной сотней различных 
прочтений). Для каждого образца определен набор потенциальных вариантов. Далее путем объединения 
составлен общий набор потенциальных вариантов всех образцов. Далее для каждого варианта в общем 
списке для каждого образца подсчитан процент прочтений, которые свидетельствуют о наличии варианта.
Результаты подсчета процентов чтений сведены в таблицу. По колонкам идут разные образцы. По строкам - потенциальные варианты. В столбце count указано, сколько раз этот вариант был реально задетектирован в 1000 других образцов. В ячейках - процент чтений, которые подтверждают наличие конкретного варианта в конкретном образце.

Поскольку везде секвенировалась кровь, в идеальной картине для настоящего варианта
должно быть либо 50% прочтений, свидетельствующих о его наличии, либо 100% (соответственно, либо 
гетерозигота, либо гомозигота - когда у человека вариант есть только одной хромосоме или на обоих 
хромосомах). В реальности эти значения могут отличаться от 50% и 100% за счет ошибок секвенирования 
и для гетерозиготы обычно могут варьироваться в пределах от 5% до 95% в зависимости от степени 
ошибки (но для гомозиготы это значение не может падать ниже 98%).
Основная доля ячеек соответствует артефактам секвенирования - то есть, в действительности варианта 
нет в образце.

Среди 15 образцов нет родственников. Иначе говоря, если вариант редко детектируется в общей 
популяции (столбец count), то он должен также редко детектироваться и среди этих 15 образцов.
Каждый вариант должен удовлетворять закону Харди-Вайнберга.То есть, вариант не может всегда 
обнаруживаться только в гетерозиготе.

Ожидаемое количество вариантов в одном образце - от 20 до 30 (что соответствует
средней вариабельности генов BRCA1/2, ATM, CHEK2 в популяции).

### Google sheet (by hand)

https://docs.google.com/spreadsheets/d/1QKV6NaBe64N5yqWBlB8CPUoTZAC5LMGKkkKgXwhHkcc/edit?usp=sharing

In [10]:
# Load data

task2 = pd.read_csv('SNV_main.tsv', sep='\t', header= 0)

In [11]:
task2

,Unnamed: 0,isPathogenic,count,isRejected,S13,S7,S8,S1,S2,S3,S5,S9,S10,S11,S12,S14,S15,S16,S17
0,chr11:108098460A>G,0,0,unwanted consequence,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,chr11:108115463T>C,0,0,unwanted consequence,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01
2,chr11:108115481T>G,0,0,unwanted consequence,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,chr11:108115661G>A,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00
4,chr11:108117678T>C,0,0,unwanted consequence,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,chr11:108098459TAA>T,0,552,high EXAC frequency (threshold 0.3%),0.95,0.49,0.66,0.51,0.99,0.51,0.73,0.46,0.99,0.46,0.03,0.11,0.90,0.11,0.46
216,chr13:32913055A>G,0,636,high EXAC frequency (threshold 0.3%),1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
217,chr13:32915005G>C,0,640,high EXAC frequency (threshold 0.3%),1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
218,chr13:32929387T>C,0,650,high EXAC frequency (threshold 0.3%),1.00,1.00,1.00,1.00,1.00,1.00,0.99,1.00,1.00,0.99,1.00,1.00,1.00,1.00,1.00


# Functions

In [254]:
# Hypergeometric test

def hypertest(index, column_name):


    # Parameters
    N_total = 1000
    N_variant_total = task2.at[index, "count"]
    n_sample = 100
    n_variant_sample = task2.at[index, column_name] * 100

    # Perform hypergeometric test (survivial function)
    p_value = hypergeom.sf(n_variant_sample - 1, N_total, N_variant_total, n_sample)
    
    # Interpret the results
        
    # P value adjusted to 0.1 from 0.05    
    if p_value < 0.1:
        #Reject
        res1 = 1
    else:
        #Cannot reject
        res1 = 0
        
    return res1

In [255]:
def classify_sample_variants(column_name):
    total_counts = len(t2[column_name])
    variant_count = t2[column_name].sum()
    value = (variant_count / total_counts) * 100
    

    if 20 <= value <= 30:
        res1 = 'Expected number of Variants (' + str(value) + ' %)'
    else:
        res1 = 'Abnormal number of variants (' + str(value) + ' %)'
        
    t2.loc[t2[column_name].last_valid_index() + 1, [column_name]] = res1

In [253]:
def HW_improvised_test():

    # Empty list to store percents
    numbers = []
    t2['H-W improvised test'] = None

    # Iterate over rows of the first dataframe
    for idx, row in t2[sample_columns].iterrows():
        # Check for values equal to 1 in the row
        for col in t2[sample_columns].columns:
            if row[col] == 1:
                # Extract corresponding value from the second dataframe
                number = task2.at[idx, col]
                # Append the extracted number to the list
                numbers.append(number)


        # Categorize variant
        new_array = ['A' if x > 0.98 else 'B' if 0.05 <= x <= 0.95 else 'C' for x in numbers]
        # A is homo recessive
        # B is hetero
        # C unknown/mistake

        if 'A' in set(new_array) and 'B' in set(new_array):
            res = "Doesn't violate HW"
        elif 'A' in set(new_array) or 'B' in set(new_array):
            res = "Violates HW"
        elif 'C' in set(new_array):
            res = "Unknown/Sequencing Mistake"
        else:
            res = ""

        t2.loc[idx, "H-W improvised test"] = res

        numbers = []

# Full script

In [259]:
#Create a working dataframe

# Filter column names from the original dataframe that start with 'A'
column_names_sample = [col for col in task2.columns if col.startswith('S')]

# Create a new dataframe
t2 = pd.DataFrame(task2.iloc[:, 0])

# Add columns from the original dataframe that start with 'A' to the new dataframe
for column_name in column_names_sample:
    t2[column_name] = task2[column_name]

In [260]:
# Script


# Iterate over columns in the original dataframe
for column_name in task2.columns:
    if column_name.startswith('S'):
        
        for index, value in task2[column_name].items():
            
            #Hypergeometric test
            t2.loc[index, column_name] = hypertest(index, column_name)        
        
        classify_sample_variants(column_name)
        
# Add sum of the detected variants
sample_columns = t2.columns[t2.columns.str.startswith('S')]
t2['Sum of Detected Variant'] = t2[sample_columns].iloc[:-1].sum(axis=1)

# Add my own bad HW test
HW_improvised_test()

In [261]:
t2

,Unnamed: 0,S13,S7,S8,S1,S2,S3,S5,S9,S10,S11,S12,S14,S15,S16,S17,Sum of Detected Variant,H-W improvised test
0,chr11:108098460A>G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,chr11:108115463T>C,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,Unknown/Sequencing Mistake
2,chr11:108115481T>G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,chr11:108115661G>A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,Unknown/Sequencing Mistake
4,chr11:108117678T>C,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,Unknown/Sequencing Mistake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,chr13:32913055A>G,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,15.0,Violates HW
217,chr13:32915005G>C,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,15.0,Violates HW
218,chr13:32929387T>C,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,15.0,Violates HW
219,chr11:108183167A>G,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,Violates HW


In [262]:
t2.to_csv('results2.tsv', index=False)
t2.to_csv('results2.csv', index=False)